## Annual Report Data Engineering and Analysis

## Setup

In [2]:
#--------------------------------------------------------------------------------------------------------#
# import packages and modules
# base packages
import os
import sys
import logging
from datetime import datetime
import pandas as pd
# external connection packages
import requests
from boxsdk import Client, CCGAuth
import sqlalchemy as sa
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
# ESRI packages
import arcpy
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor
# email packages
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# set overwrite to true
arcpy.env.overwriteOutput = True
arcpy.env.workspace = "C:\GIS\Scratch.gdb"

# in memory output file path
wk_memory = "memory" + "\\"
# set workspace and sde connections 
working_folder = "C:\GIS"
workspace      = "C:\GIS\Scratch.gdb"

# network path to connection files
filePath = "C:\\GIS\\DB_CONNECT"
# database file path 
sdeBase = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
# Feature dataset to unversion and register as version
fdata = sdeCollect + "\\sde_collection.SDE.Parcel"
# string to use in updaetSDE function
sdeString  = fdata + "\\sde_collection.SDE."
# local path to stage csvs in
accelaFiles = "//trpa-fs01/GIS/Acella/Reports"

# connect to bmp SQL dataabase
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql14;DATABASE=tahoebmpsde;UID=sde;PWD=staff"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)

# Box API credentialsn setup with CCGAuth
auth = CCGAuth(
  client_id     = "pusxamhqx4urav2lj847darrr1niydzp",
  client_secret = "tmnxqxp8sSY6i24OPX2bAYFrnIA3cerZ",
  user          = "21689880902"
)
# setup client for BOX connection
client = Client(auth)

##--------------------------------------------------------------------------------------#
## EMAIL and LOG FILE SETTINGS ##
##--------------------------------------------------------------------------------------#
## LOGGING SETUP
# Configure the logging
log_file_path = os.path.join(working_folder, "Parcel_Tables_to_Features_Log.log")  
# setup basic logging configuration
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    filename=log_file_path,  # Set the log file path
                    filemode='w')
# Create a logger
logger = logging.getLogger(__name__)
# Log start message
logger.info("Script Started: " + str(datetime.datetime.now()) + "\n")

## EMAIL SETUP
# path to text file
fileToSend = log_file_path
# email parameters
subject = "Parcel Tables to Parcel Features ETL"
sender_email = "infosys@trpa.org"
# password = ''
receiver_email = "GIS@trpa.gov"



In [ ]:

# get the bigger accela report data via the API ESA setup on LTinfo
def getAccelaLTinfoFiles(csvDict):
    for csvName,api_url in csvDict.items():
        # Send a GET request to the API
        response = requests.get(api_url)
        if response.status_code == 200:
            # If the request is successful, save the CSV content to a file
            with open(os.path.join(accelaFiles,csvName), "wb") as csv_file:
                csv_file.write(response.content)
            logger.info(f"CSV file saved as {csvName}")
        else:
            logger.info(f"Failed to fetch data from the API. Status code: {response.status_code}")

In [ ]:
# start timer for the get data requests
startTimer = datetime.datetime.now()

# dictionary of acella reports from ltinfo (check with ESA for updates or issues to their API)
ltinfoDict = {'Accela_Parcels.csv'         : 'https://laketahoeinfo.org/Api/GetAccelaParcelsCsv/1A77D078-B83E-44E0-8CA5-8D7429E1A6B4',
              'Accela_Record_Details.csv'  : 'https://laketahoeinfo.org/Api/GetAccelaRecordDetailsCsv/1A77D078-B83E-44E0-8CA5-8D7429E1A6B4',
            #   'Accela_Record_Documents.csv': 'https://laketahoeinfo.org/Api/GetAccelaRecordDocumentsCsv/1A77D078-B83E-44E0-8CA5-8D7429E1A6B4'
             }

# function to save Accela Reports from LTinfo API
getAccelaLTinfoFiles(ltinfoDict, accelaFiles, logger)


# dfSecurity = pd.read_csv(os.path.join(accelaFiles, 'Accela_Security.csv'))
# dfAParcel  = pd.read_csv(os.path.join(accelaFiles, 'Accela_Parcels.csv'))
dfAPermit  = pd.read_csv(os.path.join(accelaFiles, 'Accela_Record_Details.csv'),   on_bad_lines='skip')
# dfADoc     = pd.read_csv(os.path.join(accelaFiles, 'Accela_Record_Documents.csv'), on_bad_lines='skip')

#### Reports to add to the Batch Engine
* ReviewCompleteness_30Day_V1
* DetailHistory_Reporting

### Count of Permit by Category
*  Get Accela Permits data summarized by count of Reporting Category 
	 - Delete TMP files
	 - Establish Reporting Category
		- Summary of Record Types
			- Get lookup list for Reporting Category by Recor Type
			
		- Filter out Plan Revision
			- A second lookup of the ID without -01 at the end and lookup list against the other File Number Record Type
			
	- Count of Files by Reporting Category
		- Just for current year


In [ ]:
df = pd.read_csv()

### Tree Permits 
* setup Tree Permit Activity report in Batch Engine workflow
    - filter by year
    - merge with the main Accela permit csv by File Number
    - Tree Total
    - CHECKED in a column is the reason the tree was removed 
    - Tree Total is the count of trees that are approved to be removed
    - Application is APPROVED
- How many trees removed apps by year
- Total trees approved by permit
- Get Tree Total by Reason
- just for current year
- what % of applications have x reason CHECKED
- Created By BBARR are the Fire Districts creating a permit on our behalf
    - to look at the ones we've processed filter out the BBARR ones
        - we just want to report on TRPA actions not the Fire district issued permits

In [ ]:
parcelHistory = 
parcelMaster =

OldAPN = 
CurrentAPN = 

### Banked Development Rights Analysis

* All Banked development rights by type, land capability, location and jurisdiction
* Current Banked development rights by type, land capability, location and jurisdiction
    * group by High Capability, Low Capability, and SEZ
    * By location to town center
    * banked before and after 12/12/12

* Remove things that have been banked in the past year from Existing Development on the ground (TAU, RES, CFA)
- quanity that was removed should be subtracted from existing development
- newly built allocations (comes from transacted data in LTinfo) 
    - this falls apart when we dont have current data from the jurisdictions on allocated development
- opportunity to get data from GIS service....

## IPES Score of 0 = SEZ, IPES 1-725 = Low Capability, and IPES>726 = High Capability

#### "F:\Research and Analysis\Reporting\Annual Reports\2022 Annual Report\2022 Annual Report Data\2022 Transfer Analysis.xlsx"

In [11]:
dfDRTrans  = pd.read_json("https://laketahoeinfo.org/WebServices/GetTransactedAndBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfDRTrans.RecordType.unique()

array(['Allocation Assignment Receiving Parcel',
       'Allocation Assignment Sending Parcel',
       'Allocation Receiving Parcel', 'Banked Commodity',
       'Conversion Receiving Parcel', 'Conversion Sending Parcel',
       'Conversion With Transfer Receiving Parcel',
       'Conversion With Transfer Sending Parcel',
       'ECM Retirement Sending Parcel',
       'Land Bank Acquisition Sending Parcel',
       'Land Bank Transfer Receiving Parcel',
       'Shorezone Allocation Receiving Parcel',
       'Transfer Receiving Parcel', 'Transfer Sending Parcel'],
      dtype=object)

In [14]:
dfDRTrans.RecordType.unique()

array(['Allocation Assignment Receiving Parcel',
       'Allocation Assignment Sending Parcel',
       'Allocation Receiving Parcel', 'Banked Commodity',
       'Conversion Receiving Parcel', 'Conversion Sending Parcel',
       'Conversion With Transfer Receiving Parcel',
       'Conversion With Transfer Sending Parcel',
       'ECM Retirement Sending Parcel',
       'Land Bank Acquisition Sending Parcel',
       'Land Bank Transfer Receiving Parcel',
       'Shorezone Allocation Receiving Parcel',
       'Transfer Receiving Parcel', 'Transfer Sending Parcel'],
      dtype=object)

In [9]:
dfDRBank   = pd.read_json("https://laketahoeinfo.org/WebServices/GetBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfDRBank

,APN,DevelopmentRight,LandCapability,IPESScore,CumulativeBankedQuantity,RemainingBankedQuantity,Jurisdiction,LocalPlan,DateBankedOrApproved,HRA,LastUpdated
0,014-303-002,Potential Residential Unit of Use (PRUU),IPES,733.0,1,1,"El Dorado County, CA (ELDO)",Tahoma Residential,12/03/2018,McKinney Bay,12/03/2018
1,015-103-008,Potential Residential Unit of Use (PRUU),IPES,0.0,1,1,"El Dorado County, CA (ELDO)",TAHOMA RESIDENTIAL,04/03/2006,McKinney Bay,10/13/2017
2,015-113-003,Potential Residential Unit of Use (PRUU),Bailey 1b,NaN,1,1,"El Dorado County, CA (ELDO)",TAHOMA RESIDENTIAL,09/03/2015,McKinney Bay,10/16/2017
3,015-154-012,Potential Residential Unit of Use (PRUU),Bailey 1b,NaN,1,1,"El Dorado County, CA (ELDO)",TAHOMA RESIDENTIAL,12/17/2003,McKinney Bay,10/11/2017
4,015-204-019,Potential Residential Unit of Use (PRUU),IPES,0.0,1,1,"El Dorado County, CA (ELDO)",TAHOMA RESIDENTIAL,06/24/2002,McKinney Bay,10/17/2017
...,...,...,...,...,...,...,...,...,...,...,...
1515,1418-34-401-028,Coverage (hard),Bailey 2,NaN,109,109,"Douglas County, NV (DGCO)",LAKERIDGE,09/11/2020,Cave Rock,09/20/2021
1516,1418-34-401-028,Coverage (hard),Bailey 4,NaN,1856,1856,"Douglas County, NV (DGCO)",LAKERIDGE,09/11/2020,Cave Rock,09/20/2021
1517,1418-34-401-028,Single-Family Residential Unit of Use (SFRUU),Bailey 6,NaN,1,1,"Douglas County, NV (DGCO)",LAKERIDGE,06/10/2013,Cave Rock,09/20/2021
1518,1418-34-601-008,Coverage (hard),Bailey 2,NaN,28,28,"Douglas County, NV (DGCO)",LAKERIDGE,12/04/2013,Cave Rock,03/12/2021


In [8]:
dfDRBank.DevelopmentRight.unique()

array(['Potential Residential Unit of Use (PRUU)', 'Coverage (hard)',
       'Single-Family Residential Unit of Use (SFRUU)', 'Coverage (soft)',
       'Commercial Floor Area (CFA)', 'Tourist Accommodation Unit (TAU)',
       'Restoration Credit', 'Residential Floor Area (RFA)',
       'Coverage (potential)', 'Residential Allocation',
       'Residential Bonus Unit (RBU)', 'Tourist Floor Area (TFA)',
       'Multi-Family Residential Unit of Use (MFRUU)',
       'Persons-at-one-time (PAOT)'], dtype=object)